# ADAS vs HDV impact on platoons

This code is the reposit corrsponding to the ATEC presentation "Analyse de l’impact de la présence de systèmes ADAS sur les véhicules en formation de peloton"<br>


In [50]:
#imports for platoon identification
import src.read_data as read
import src.test_CF as CF
import src.test_platoon as platoons


#imports for platoon analysis
import src.analyze_platoons_life
import src.compare_DTW_ACC_HDV as DTWimpact
import src.compute_half_life as half_life
import src.detect_string_instability_platoon as instability_detection

import src.examine_string_instability  as instability_analysis


#misc imports 
import multiprocessing
import os
import ast
import pandas as pd
import numpy as np

## Preprocess data

You can bypass this cell if you just want to conduct the analysis

In [ ]:
print('declare path of the downloaded trajectory data')
df_path = input()
print('declare path of the trajectory data')
path_out = input()
read.clean_data(df_path, path_out)

The path "path_out", is the one that you will use in the remaining of this notebook when asked to declare a path for trajecory data

## Identifying the platoons

Those two cells underneath allows for the detection of the platoons in the trajectory dataset. They are time consumming and can be bypassed 

In [1]:
#identify vehicles Car-following
if __name__ == "__main__":
    print('declare path of the trajectory data')
    path = input()
    window = 30
    step = 5
    tau_values = [0.6 + 0.1 * k for k in range(0, 14)]
    
    files = [(file, path, tau_values, window, step) for file in os.listdir(path)]

    num_workers = multiprocessing.cpu_count() 
    with multiprocessing.Pool(processes=num_workers) as pool:
        pool.map(CF.process_file, files)

In [ ]:
#identify platoons
if __name__ == "__main__":
    print('define trajectories path')
    path = input()
    dtw_path = 'out/DTW/'
    dtw_threshold = 1.5
    files_info = [(file, dtw_path, path, dtw_threshold) for file in os.listdir(path)]
    num_workers = multiprocessing.cpu_count()  
    with multiprocessing.Pool(processes=num_workers) as pool:
        pool.map(platoons.routine, files_info)

for f in os.listdir('out/string_dtw/'):
    df = pd.read_csv('out/string_dtw/'+f)
    platoondf = platoons.study_platoon_evolution(df)
    platoondf.to_csv('out/platoons/'+f)

## preparing data for the analysis

This cell is to be used if you used the two previous cells. If you are using this code just for the analysis of the platoons. You can bypass it

In [ ]:
print('define trajectories path')
trajspath = input()
platoonspath = 'out/platoons/'
ACC_list= []
for f in os.listdir(platoonspath):
    platoon = pd.read_csv(os.path.join(platoonspath, f))
    platoon['platoon compo'] = platoon['platoon compo'].apply(ast.literal_eval)
    trajs = pd.read_csv(os.path.join(trajspath, f))
    ACC_set = set(trajs[trajs['ACC'] == 'Yes']['ID'].unique())
    platoon['ACC'] = platoon['platoon compo'].apply(lambda k: [l in ACC_set for l in k])

## Analysis of the impact of the presence of ADAS vehicles in a platoon

### Bypass to this cell if you just want the data analysis

In [1]:
#outputs paths
datapath = 'data/by_run/corrected/'
DTWpath = 'out/string_dtw/'
platoonspath = 'out/platoons/'
import src.compare_DTW_ACC_HDV as DTWimpact
import src.compute_half_life as half_life

In [ ]:
#load data for life expectancy analysis
data_platoons = half_life.import_platoons(platoonspath)

data_no_ACC = data_platoons[data_platoons['has_ACC'] == False]
data_no_ACC = half_life.count_platoons_per_duration(data_no_ACC)
data_no_ACC['log_remaining_platoons'] = np.log(data_no_ACC['number of platoons'])

data_ACC = half_life.count_platoons_per_duration(data_platoons)
data_ACC['log_remaining_platoons'] = np.log(data_ACC['number of platoons'])


In [ ]:
model_no_ACC = half_life.perform_statistical_analysis(data_no_ACC, 'No ACC')
model_ACC = half_life.perform_statistical_analysis(data_ACC, 'ACC')

In [ ]:
intercept = model_ACC.params['const']
slope_ACC = model_ACC.params['duration']
conf_int_ACC = np.exp(model_ACC.conf_int().loc['duration'].values)

half_life.plot_results(data_ACC, data_no_ACC, intercept, slope_ACC, conf_int_ACC)

In [ ]:
print('half_life HDV platoons')
half_life.compute_half_life(model_no_ACC)
print('half_life all platoons')
half_life.compute_half_life(model_ACC)

In [ ]:
#analyze the impact of an ADAS on DTW values
compile = DTWimpact.load_and_process_data(datapath, DTWpath)
DTWimpact.filter_and_plot_data(compile)
DTWimpact.perform_statistical_tests(compile)

In [ ]:
#examien impact of ADAS on string instability
instability_detection.routine(platoonspath,datapath)


In [ ]:
df = instability_analysis.import_platoons(platoonspath)
df['platoon compo'] = df['platoon compo'].apply(ast.literal_eval)
df['length'] = df['platoon compo'].apply(len) + 1
df = df[df['length']<6] 

instability_analysis.plot_stability(df)
instability_analysis.examine_stats_instability(df)